# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set our OpenAI, Tavily, and LangSmith API keys along with our LangSmith environment variables.

In [55]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [57]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

TAVILY_API_KEY ········


In [59]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE8 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

LangSmith API Key:  ········


## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [61]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [64]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [67]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

ANSWER✅

The model determines which tool to use by considering both the user prompt, which is the question or instruction entered by the person, and the internal built-in prompts, which are behind-the-scenes instructions that guide how to interpret and act on the user’s request. It’s this coordination between what the user asks for and how the system is internally guided that allows the model to “think” and decide the best tool to use.

For example, if the prompt is “What is the temperature in Austin right now?”, the model recognizes that this is a real-time data request. It may choose a weather API tool to fetch current temperature data, because the user wants live, factual information. However, if the prompt is “Summarize the latest news about Austin traffic”, the model recognizes a text processing task. It may first use a news-fetching tool to collect articles and then pass that text to a summarization tool to condense it into a clear summary.

These two example tasks are different because one requires retrieving factual data from an external source, while the other requires processing and understanding text before delivering a summary. The model decides which tools to use based on the type of task implied by the user prompt and the guidance from its internal instructions.




## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [8]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [9]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [10]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [11]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [12]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [13]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

ANSWER ✅
There isn’t a built-in limit to how many times a multi-agent system can cycle. While it could keep iterating as long as the system allows, this can be problematic because it may lead to infinite loops, wasted resources, and slower performance. Setting a limit is important to ensure the system operates efficiently, prevents excessive computation, and avoids getting stuck in repetitive cycles without useful results.


If not, how could we impose a limit to the number of cycles?

ANSWER ✅
There are multiple ways to limit the number of cycles, which can be used individually or in combination:

    1.) Global Cycle Counter – Track the total number of system-wide cycles and set a maximum allowed number. Once this limit is reached, the system stops.
    2.) Agent-Specific Iteration Caps – Each agent tracks its own execution count and stops once it reaches a defined threshold. Some agents may finish sooner than others, providing flexibility.
    3.) Overall Run Time – Implement a time-based limit so the system stops after a specified duration, independent of the number of cycles.
    4.) Specific Conditions – Stop cycles based on output criteria, such as if the results haven’t changed significantly or if a specific goal hasn’t been achieved.

Overall, these mechanisms provide flexibility to ensure the system runs efficiently while preventing infinite loops or unnecessary computation.



## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [14]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="How are technical professionals using AI to improve their work?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='Technical professionals are using AI in various ways to enhance their work, including automating repetitive tasks, improving decision-making, analyzing large datasets, developing new products and services, and optimizing processes. They leverage AI for tasks such as machine learning model development, natural language processing, computer vision, predictive analytics, and automation. This integration helps increase efficiency, accuracy, and innovation across different industries. Would you like specific examples from particular fields or recent trends?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 163, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-1

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [16]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the A Comprehensive Survey of Deep Research paper, then search each of the authors to find out where they work now using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eWPkp6AZXUZ8Ddv5RSccsgOD', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_inftH8lgkIIRT3PGEwdzfSTn', 'function': {'arguments': '{"query": "author of A Comprehensive Survey of Deep Research"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 182, 'total_tokens': 242, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CJ4iCWQU8LgioK1yxump6CTPubRVg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--72e6fe53-5b0

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

ANSWER✅

1.) Request Input – The user’s request was defined and the state object was populated. In this example, the request was to search for a specific research paper, extract the authors’ names, and then find out where they currently work.

2.) Agent Node Decision – The state object was passed into the agent node (entry point). The agent node added an AIMessage to the state object and determined which tools to use. Here, Arxiv was used to retrieve the paper and author names, and Tavily was used to look up the authors’ current workplaces.

3.) Action Nodes Execute Tools – Each action node executed its respective tool. The responses from Arxiv and Tavily were added to the state object and sent back along the conditional edge to the agent node.

4.)Agent Node Updates – The agent node processed the tool responses, added a summary of the authors’ workplaces to the state object, and passed it along the conditional edge.

5.) Conditional Edge & END – The conditional edge received the state object, found no further “tool_calls” required, and passed the state object to END.

6.) Output – The final state object contains the original request, all tool responses, and the summarized author information. Each chunk in the output reflects updates from a specific node.

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [96]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["text"])]}

def parse_output(input_state):
  return {"answer" : input_state["messages"][-1].content}

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

agent_chain_with_formatting.invoke({"text" : "What is Deep Research?"})

{'answer': 'Deep Research typically refers to an in-depth and comprehensive investigation or analysis into a specific topic, subject, or field. It involves gathering detailed information, examining various sources, and analyzing data thoroughly to gain a profound understanding of the subject matter. Deep Research is often used in academic, scientific, technological, and business contexts to develop insights, inform decision-making, or advance knowledge.\n\nIf you are referring to a specific organization, product, or service named "Deep Research," please provide more context so I can give a more precise answer.'}

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    {
        "inputs" : {"text" : "Who were the main authors on the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' paper?"},
        "outputs" : {"must_mention" : ["Peng", "Xu"]}   
    },
    ...,
    {
        "inputs" : {"text" : "Where do the authors of the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' work now?"},
        "outputs" : {"must_mention" : ["Zhejiang", "Liberty Mutual"]}
    }
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions that pertain to the cohort use-case (more information [here](https://www.notion.so/Session-4-RAG-with-LangGraph-OSS-Local-Models-Eval-w-LangSmith-26acd547af3d80838d5beba464d7e701#26acd547af3d81d08809c9c82a462bdd)), or the use-case you're hoping to tackle in your Demo Day project.

In [121]:

##My demo day project idea as of now: Demo Day Project – Event Evaluator: A multi-agent application that analyzes Northern VA traffic conditions by pulling data from news sources and real-time incident APIs. 
##It identifies the causes of congestion, summarizes incidents, and generates comprehensive reports or speeches, helping city planners, commuters, and organizations understand why traffic occurred and communicate insights effectively.



questions = [
    {
        "inputs": {"text": "What caused the traffic congestion on I-95 General Purpose lanes in GWA region yesterday morning?"},
        "outputs": {"must_mention": ["accident", "construction", "weather", "incident", "special event"]}
    },
    {
        "inputs": {"text": "Which areas in Northern Virginia had the longest commute times yesterday and why?"},
        "outputs": {"must_mention": ["Arlington", "Fairfax", "Alexandria", "Tysons", "delays"]}
    },
    {
        "inputs": {"text": "Summarize yesterday's major traffic incidents for a local news bulletin."},
        "outputs": {"must_mention": ["accidents", "road closures", "highways", "emergency response"]}
    },
    {
        "inputs": {"text": "Write a brief speech for city council explaining why traffic was unusually heavy yesterday."},
        "outputs": {"must_mention": ["public safety", "traffic management", "incident highlights"]}
    },
    {
        "inputs": {"text": "Create a commuter alert explaining alternative routes to avoid yesterday's traffic delays."},
        "outputs": {"must_mention": ["alternative routes", "delays", "highways", "traffic updates"]}
    },
    {
        "inputs": {"text": "Analyze yesterday's traffic events to suggest improvements for emergency response."},
        "outputs": {"must_mention": ["response times", "incident handling", "recommendations", "traffic flow"]}
    }
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [103]:
from langsmith import Client

client = Client()

dataset_name = f"Simple Search Agent - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about potential demo day idea to evaluate the Simple Search Agent."
)

client.create_examples(
    dataset_id=dataset.id,
    examples=questions
)

{'example_ids': ['53fea113-6752-498f-a55c-30588cde8fc9',
  'a008d2ab-de04-4385-ad8c-ee7344a4ac34',
  '207b98dc-f3bb-4e00-9626-e0f0bbba4fc3',
  '2aeb386b-849e-4799-8967-7159c006d640',
  '4dabbc28-f52c-4ace-a1bf-2e9b63791fb8',
  '63f65d04-ecc8-4a9f-bcbb-8fa6e5501361'],
 'count': 6}

### Task 2: Adding Evaluators

Let's use the OpenEvals library to product an evaluator that we can then pass into LangSmith!

> NOTE: Examine the `CORRECTNESS_PROMPT` below!

In [106]:
from openevals.prompts import CORRECTNESS_PROMPT
print(CORRECTNESS_PROMPT)

You are an expert data labeler evaluating model outputs for correctness. Your task is to assign a score based on the following rubric:

<Rubric>
  A correct answer:
  - Provides accurate and complete information
  - Contains no factual errors
  - Addresses all parts of the question
  - Is logically consistent
  - Uses precise and accurate terminology

  When scoring, you should penalize:
  - Factual errors or inaccuracies
  - Incomplete or partial answers
  - Misleading or ambiguous statements
  - Incorrect terminology
  - Logical inconsistencies
  - Missing key information
</Rubric>

<Instructions>
  - Carefully read the input and output
  - Check for factual accuracy and completeness
  - Focus on correctness of information rather than style or verbosity
</Instructions>

<Reminder>
  The goal is to evaluate factual correctness and completeness of the response.
</Reminder>

<input>
{inputs}
</input>

<output>
{outputs}
</output>

Use the reference outputs below to help you evaluate the

In [108]:
from openevals.llm import create_llm_as_judge

correctness_evaluator = create_llm_as_judge(
        prompt=CORRECTNESS_PROMPT,
        model="openai:o3-mini", # very impactful to the final score
        feedback_key="correctness",
    )

Let's also create a custom Evaluator for our created dataset above - we do this by first making a simple Python function!

In [111]:
def must_mention(inputs: dict, outputs: dict, reference_outputs: dict) -> float:
  # determine if the phrases in the reference_outputs are in the outputs
  required = reference_outputs.get("must_mention") or []
  score = all(phrase in outputs["answer"] for phrase in required)
  return score

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

ANSWER ✅

The initial evaluation metric is a good starting point but primarily measures keyword recall and factual reporting. It could be improved by diversifying question types to include analytical and predictive tasks, adding granularity to expected outputs (e.g., numeric values or time ranges), and introducing ambiguity to test reasoning under uncertainty. While the current evaluation relies heavily on keywords, I should broaden the keyword set and add tests that better assess reasoning ability and output complexity.



Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [115]:
results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset.name,
    evaluators=[correctness_evaluator, must_mention],
    experiment_prefix="simple_agent, baseline",  # optional, experiment name prefix
    description="Testing the baseline system.",  # optional, experiment description
    max_concurrency=4, # optional, add concurrency
)

View the evaluation results for experiment: 'simple_agent, baseline-2a219cdf' at:
https://smith.langchain.com/o/bf3a953b-d5ae-479e-b06a-74d94431c40b/datasets/b5a20e55-36b9-40b5-82f0-d12b044e05f9/compare?selectedSessions=94858c9c-fd31-45c8-907b-0b541a32bb33




0it [00:00, ?it/s]

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [118]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #4:

Please write markdown for the following cells to explain what each is doing.

## Creating an Enhanced State Graph

This cell creates a new  instance called  that will include an additional helpfulness evaluation step. This graph will be similar to our previous agent but with an added quality control mechanism to ensure the agent's responses are truly helpful to the user.

The graph starts with the same two core nodes:
- : The main reasoning node that calls the language model
- : The tool execution node that runs any required tools


In [20]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

## Setting the Entry Point

This cell establishes that our enhanced agent will start with the  node, just like our previous simple agent. The agent will begin by processing the user's input and deciding whether to use tools or provide a direct response.


In [21]:
graph_with_helpfulness_check.set_entry_point("agent")

## Implementing the Helpfulness Check Logic

This cell defines the  function, which is the key enhancement to our agent. This function serves as a conditional router that:

1. **Checks for tool calls**: If the agent wants to use tools, it routes to the "action" node
2. **Implements a helpfulness evaluation**: If no tools are needed, it evaluates whether the response is truly helpful
3. **Prevents infinite loops**: Stops execution after 10 message exchanges to avoid endless conversations
4. **Uses a separate AI model**: Employs GPT-4.1-mini to objectively evaluate response quality

The helpfulness check compares the original user query with the agent's final response and asks a specialized model to determine if the response adequately addresses the user's needs (Y for helpful, N for unhelpful).


In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

## Configuring the Decision Logic

This cell adds the conditional routing logic to our graph. The  method connects the "agent" node to different paths based on the  function's output:

- **"continue" → "agent"**: If the response isn't helpful enough, the agent tries again
- **"action" → "action"**: If tools are needed, execute them
- **"end" → END**: If the response is helpful, conclude the conversation

This creates a feedback loop that allows the agent to refine its responses until they meet quality standards.


In [23]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

## Connecting Action Back to Agent

This cell ensures that after executing tools in the "action" node, the agent returns to the "agent" node to process the tool results and potentially generate a final response. This maintains the flow where tool execution is followed by reasoning about the results.


In [24]:
graph_with_helpfulness_check.add_edge("action", "agent")

## Compiling the Enhanced Agent

This cell compiles our enhanced state graph into an executable agent called . The compilation process optimizes the graph structure and prepares it for execution with all the conditional logic and helpfulness evaluation in place.


In [25]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

## Testing the Helpfulness-Enhanced Agent

This cell demonstrates the enhanced agent in action by asking "What are Deep Research Agents?" The agent:

1. **Processes the query** through the main agent node
2. **Evaluates its own response** using the helpfulness check
3. **Decides whether to continue or end** based on response quality
4. **Streams the results** showing the agent's reasoning process

The output shows that the agent provided a comprehensive response about Deep Research Agents and the helpfulness check determined it was satisfactory, so the conversation ended appropriately.


In [27]:
inputs = {"messages" : [HumanMessage(content="What are Deep Research Agents?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='Deep Research Agents are advanced AI systems designed to assist with in-depth research tasks. They leverage deep learning techniques and large datasets to analyze complex information, generate insights, and support decision-making across various fields such as science, technology, medicine, and more. These agents can automate literature reviews, extract relevant data from vast sources, and provide comprehensive summaries, making research more efficient and thorough. Would you like me to find more detailed or specific information about Deep Research Agents?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 158, 'total_tokens': 251, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-

## Part 3: LangGraph for the "Patterns" of GenAI

### Task 4: Helpfulness Check of Gen AI Pattern Descriptions

Let's ask our system about the 3 main patterns in Generative AI:

1. Context Engineering
2. Fine-tuning
3. Agents

In [28]:
patterns = ["Context Engineering", "Fine-tuning", "LLM-based agents"]

In [29]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Context Engineering is a relatively new interdisciplinary field that focuses on designing, managing, and optimizing the context in which systems, especially artificial intelligence and software applications, operate. It involves understanding and shaping the environment, circumstances, and background information that influence how systems behave and interact with users. The goal is to improve system performance, user experience, and decision-making by carefully engineering the context.

The concept of Context Engineering has gained prominence with the rise of AI, IoT, and complex software systems, where context plays a crucial role in system effectiveness. It started to break onto the scene in the late 2010s and early 2020s, driven by advancements in contextual AI, ubiquitous computing, and the need for more adaptive and personalized systems.

Would you like me to find more detailed or specific information about its origins and development?



Fine-tuning is a machine learning techniqu